
# Load Data, Drop Unecessary Information

In [ ]:
import pandas as pd
import os

In [ ]:
def create_complete_df(num_files):
    path_to_features  = '/content/drive/MyDrive/SOC/Data/'

    list_of_dfs = []
    for i in range(num_files):
        csv_file = f'SRFastFeatures.csv'
        print(csv_file)
        df = pd.read_csv(os.path.join(path_to_features, csv_file))
        list_of_dfs.append(df)

    df = pd.concat(list_of_dfs, axis=0)
    return df

In [ ]:
df = create_complete_df(1)

In [ ]:
df.reset_index(inplace = True)

In [ ]:
df.drop(['index'], axis = 1, inplace = True)

# Standardization

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
columns_except_mu_global = list(filter(lambda x: x != "MU_GLOBAL", df.columns))

df[columns_except_mu_global] = scaler.fit_transform(df[columns_except_mu_global])

# Missing Values

In [ ]:
imputed_df = df.fillna(0)

# Merge HWSD Data

In [ ]:
HWSD = pd.read_csv('/content/drive/MyDrive/SOCData/HWSD_DATA.csv')
HWSD = HWSD[['MU_GLOBAL', 'T_OC']]
HWSD

In [ ]:
HWSD = HWSD.groupby('MU_GLOBAL').mean()

In [ ]:
new = pd.merge(imputed_df, HWSD, on="MU_GLOBAL")
new = new.dropna()

# Split Data

In [ ]:
y = new.pop('T_OC')
X = new

In [ ]:
assert 'T_OC' not in X.columns

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42)

In [ ]:
assert len(X_train) == len(y_train)

In [ ]:
assert len(X_test) == len(y_test)

# Decision Tree Regressor

In [ ]:
from sklearn.tree import DecisionTreeRegressor

dt = DecisionTreeRegressor()
dt.fit(X_train, y_train)

# Random Forest Regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor()
rf.fit(X_train, y_train)

# XGBoost

In [ ]:
from xgboost import XGBRegressor

X_train_xgb, X_val, y_train_xgb, y_val = train_test_split(X_train, y_train)

xgb = XGBRegressor(n_estimators=1500, learning_rate=0.01, eval_metric="rmse", early_stopping_rounds=10)
xgb.fit(X_train, y_train, verbose =100)

# SVM

In [ ]:
from sklearn.svm import SVR

svr = SVR(C=1.0, epsilon=0.2)
svr.fit(X_train, y_train)

# LGBM

In [ ]:
import lightgbm

hyper_params = {
    'objective': 'regression',
    'metric': 'rmse',
    'learning_rate': 0.005,
    'verbose': -1,
    'n_estimators': 1000,
    'random_state' : 0
}

lgbm = lightgbm.LGBMRegressor(**hyper_params)

lgbm.fit(X_train_xgb,
         y_train_xgb,
         eval_set = [(X_val, y_val)],
         callbacks = [lightgbm.early_stopping(stopping_rounds = 20)],
         verbose = 100
)

# RMSE

In [ ]:
from sklearn.metrics import mean_squared_error

models_list = {
    "Decision Tree" : dt,
    "Random Forest" : rf,
    "XGBoost" : xgb,
    "SVR" : svr,
    "LGBM" : lgbm,
}

for name, model in models_list.items():
    print(name + ": " + str(round(mean_squared_error(model.predict(X_test), y_test, squared = False), 5)))